# Part 1 : Web Scraping

#### Import The data from Wikepedia

In [108]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df = pd.read_html(str(table))
dataset = df[0]
dataset.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


#### Create the dataframe from the html

In [109]:
headers = dataset.iloc[0]
dataset  = pd.DataFrame(dataset.values[1:], columns=headers)

In [110]:
unique_Postcode = dataset["Postcode"].unique()
datasetUniquePost = pd.DataFrame([], columns=dataset.columns.values )

for i, val in enumerate(unique_Postcode): 
    f1 = dataset[dataset['Postcode']==val]
    if len(f1) > 1 :
        newN = ""
        i=0;
        for index, row in f1.iterrows():
            if i < (len(f1)-1) : 
                newN = newN + row['Neighbourhood'] + ", " 
                i = i+1
            else :
                newN = newN + row['Neighbourhood'] 
                i = i+1
        df2 = pd.DataFrame({"Postcode":row['Postcode'],"Borough" :  row['Borough'],"Neighbourhood":newN },index=[0]) 
        datasetUniquePost = df_temp.append(df2)
    else : 
        datasetUniquePost = df_temp.append(f1) 

#### if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [111]:
withNoNeighbourhoodIndex  = datasetUniquePost[datasetUniquePost["Neighbourhood"]=="Not assigned"].index
datasetUniquePost.loc[withNoNeighbourhoodIndex,'Neighbourhood'] = datasetUniquePost.loc[withNoNeighbourhoodIndex,'Borough']

In [112]:
datasetUniquePost.shape

(104, 3)